In [138]:
import pandas as pd
import numpy as np
import re

In [139]:


pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)



In [140]:
'''
def validate_input_json(d):
    validation_error=''
    mn_price, mx_price, loc, st, prop, min_b, max_b = False, False, False, False, False, False, False
    
    for i in d.keys():
        if 'min_price' == i:
            mn_price=True
        elif 'max_price' == i:
            mx_price=True
        elif 'locations' == i:
            loc=True
        elif 'state' == i:
            st=True
        elif 'property_type' == i:
            prop=True    
        elif 'min_beds' == i:
            min_b=True    
        elif 'max_beds' == i:
            max_b=True    
        else:
            validation_error = "Invalid key found in input: "+ i + " "
            
    
    if mn_price is False:
        validation_error= validation_error + 'min_price key is missing in input json ; '
        
    if mx_price is False:
        validation_error= validation_error + 'max_price key is missing in input json ; '
        
    if loc is False:
        validation_error= validation_error + 'locations key is missing in input json ; '
    
    
    return validation_error, st,prop, min_b, max_b

'''

'\ndef validate_input_json(d):\n    validation_error=\'\'\n    mn_price, mx_price, loc, st, prop, min_b, max_b = False, False, False, False, False, False, False\n    \n    for i in d.keys():\n        if \'min_price\' == i:\n            mn_price=True\n        elif \'max_price\' == i:\n            mx_price=True\n        elif \'locations\' == i:\n            loc=True\n        elif \'state\' == i:\n            st=True\n        elif \'property_type\' == i:\n            prop=True    \n        elif \'min_beds\' == i:\n            min_b=True    \n        elif \'max_beds\' == i:\n            max_b=True    \n        else:\n            validation_error = "Invalid key found in input: "+ i + " "\n            \n    \n    if mn_price is False:\n        validation_error= validation_error + \'min_price key is missing in input json ; \'\n        \n    if mx_price is False:\n        validation_error= validation_error + \'max_price key is missing in input json ; \'\n        \n    if loc is False:\n       

In [141]:
def load_agents():
    '''
    import os
    file_name=os.environ.get('file_name', 'file_name environment variable is not set.')
    agents = pd.read_csv(file_name)
    '''
    agents = pd.read_csv('gs://nest-agents/quantarium_agents.csv')
    
    return agents


In [142]:
def match_locations_one(agents, p_user_locations, p_state, p_state_flag):
    
    error_flag= False
    error_message= ''
    
    
    if p_state_flag:
        agents.loc[ (agents['areaServed'].str.contains(p_user_locations[0], na=False)) &  (agents['state'].str.contains(p_state[0], na=False)), 'location_match'] = 1 
    else:
        agents.loc[ (agents['areaServed'].str.contains(p_user_locations[0], na=False)) , 'location_match'] = 1 
    
    
    agents= agents[ (agents["location_match"] > 0 )]
    
    if agents.shape[0] <= 0:
        error_flag=True
        error_message='No realtor found in input location(s). Please try different location(s)'
    
    
    return agents, error_flag, error_message


In [143]:
def match_locations(agents, p_user_locations, p_state, p_state_flag):
    
    error_flag= False
    error_message= ''
    
   
    '''   
    agents["locations"]= agents["areaServed"].str.replace(' ', '')
    agents["locations"]=agents["locations"].str.lower()
    ''' 
    agents["locations"]= agents["areaServed"].str.split(",")
    agents["state"]= agents["state"].str.split(",")
       
    
    hh=[]
    hh_state=[]
                      
    if p_state_flag:
        for i in range(0, len(agents)):
            hh.append(np.intersect1d(agents["locations"][i], p_user_locations).shape[0])
            hh_state.append(np.intersect1d(agents["state"][i], p_state).shape[0])

        agents["location_match"]=hh
        agents["state_match"]=hh_state

        #agents = match_state(agents, p_state)
        #agents.fillna(0,inplace=True)

        agents= agents[ (agents["location_match"] > 0 ) & (agents["state_match"] > 0)]
        
        
    else:
        for i in range(0, len(agents)):
            hh.append(np.intersect1d(agents["locations"][i], p_user_locations).shape[0])
        
        agents["location_match"]=hh
        agents= agents[ (agents["location_match"] > 0 )]
        
    if agents.shape[0] <= 0:
        error_flag=True
        error_message='No realtor found in input location(s). Please try different location(s)'
    
    return agents, error_flag, error_message


In [144]:
def match_state(agents, p_state):
    
    agents.loc[ (agents['state'].str.contains(p_state, na=False)) , 'state_match'] = 1 
    return agents

In [145]:
def match_beds(agents, p_min_beds, p_max_beds):
    
    agents.loc[ (p_min_beds >= agents.min_beds) & (p_max_beds <= agents.max_beds), 'beds_match'] = 1 
    return agents

In [146]:
def match_property_type(agents, p_property_type):
    
    agents.loc[ (agents['property_type'].str.contains(p_property_type, na=False)) , 'property_type_match'] = 1
    
    #agents.loc[ (agents.property_type == p_property_type), 'property_type_match'] = 1 
    
    return agents

In [147]:
def match_price(agents, p_min_price, p_max_price):
    
    l_min_price = p_min_price + (p_min_price * 10)/100
    l_max_price = p_min_price - (p_min_price * 10)/100

    agents.loc[ (l_min_price >= agents.min_price) & (l_max_price <= agents.max_price), 'price_match'] = 1
    return agents

In [148]:

def validate_input_json(d):
    validation_error = ''
    validation_flag  = False
    
    # Global Variables
    global g_user_id_flag, g_user_key_flag, g_locations_flag, g_state_flag, g_min_price_flag, g_max_price_flag, g_min_beds_flag, g_max_beds_flag, g_property_flag
    g_user_id_flag, g_user_key_flag, g_locations_flag, g_state_flag, g_min_price_flag, g_max_price_flag, g_min_beds_flag, g_max_beds_flag, g_property_flag= \
    False,False,False,False,False,False,False,False,False
    
    global g_user_id_value, g_user_key_value, g_locations_value, g_state_value, g_min_price_value, g_max_price_value, g_min_beds_value, g_max_beds_value, g_property_value
    g_user_id_value, g_user_key_value, g_locations_value, g_state_value, g_min_price_value, g_max_price_value, g_min_beds_value, g_max_beds_value, g_property_value= \
    '','','','',0,0,0,0,''

    for i in d.keys():
        j=i
        i=i.lower()
        if 'user_id' ==i:
            g_user_id_flag=True
            g_user_id_value = d[j]
        elif 'user_key' == i:
            g_user_key_flag=True
            g_user_key_value = d[j]
        elif 'min_price' == i:
            g_min_price_flag=True
            g_min_price_value = d[j]
        elif 'max_price' == i:
            g_max_price_flag=True
            g_max_price_value = d[j]
        elif 'locations' == i:
            g_locations_flag=True
            g_locations_value = d[j]
        elif 'state' == i:
            g_state_flag=True
            g_state_value = d[j]
        elif 'property_type' == i:
            g_property_flag=True  
            g_property_value = d[j].strip().lower()
        elif 'min_beds' == i:
            g_min_beds_flag=True 
            g_min_beds_value = d[j]
        elif 'max_beds' == i:
            g_max_beds_flag=True 
            g_max_beds_value = d[j]
        else:
            validation_error = "Invalid key found in input: "+ i + " "
            validation_flag = True
            
    if ( (g_user_id_flag is False) | (g_user_key_flag is False)):
        validation_error = 'User_Id and (or) User_Key is missing'
        validation_flag = True
        return validation_error, validation_flag
    else:
        authenticate = pd.read_json('gs://nest-agents/authentication.json',orient='columns')
        if (( authenticate['userid'].where( (authenticate['userid'] == g_user_id_value) & (authenticate['userkey'] == g_user_key_value))
           ).dropna().shape[0]
           ) == 0:
            validation_error = 'Invalid User_Id and (or) User_Key - Access denied! '
            validation_flag = True
            return validation_error, validation_flag
        
    
    if g_min_price_flag is False:
        validation_error= validation_error + 'min_price key is missing ; '
        validation_flag = True
        
    if g_max_price_flag is False:
        validation_error= validation_error + 'max_price key is missing ; '
        validation_flag = True
        
    if g_locations_flag is False:
        validation_error= validation_error + 'locations key is missing ; '
        validation_flag = True
    
    
    return validation_error, validation_flag

In [149]:
import uuid 
print (uuid.uuid1()) 

e87c712c-112d-11ea-ac32-42010a8a0004


In [150]:
import json



#data='{"user_id":"nestready","user_key":"b942c5ec-0f97-11ea-9fd7-42010a8a0004","locations":["new-york"], "state":[""],"min_price":95000,"max_price":385000, "min_beds":2, "max_beds":3, "property_type":"townhouse"}'

data='{"user_id":"nestready","user_key":"b942c5ec-0f97-11ea-9fd7-42010a8a0004","locations":"pike","state":"MS","min_price":5000,"max_price":385000, "min_beds":1, "max_beds":4, "property_type":"Other"}'


request_json = json.loads(data)

In [151]:
authenticate = pd.read_json('gs://nest-agents/authentication.json',orient='columns')

In [152]:
authenticate

,userid,userkey
0,nestready,b942c5ec-0f97-11ea-9fd7-42010a8a0004
1,nestready-temp,b942c5ec-0f97-11ea-9fd7-42010a8a00041


In [153]:

error_message, error_flag =validate_input_json(request_json)
error_message, error_flag

('', False)

In [154]:
g_user_id_flag, g_user_key_flag, g_locations_flag, g_state_flag, g_min_price_flag, g_max_price_flag, g_min_beds_flag, g_max_beds_flag, g_property_flag

(True, True, True, True, True, True, True, True, True)

In [155]:
g_user_id_value, g_user_key_value, g_locations_value, g_state_value, g_min_price_value, g_max_price_value, g_min_beds_value, g_max_beds_value, g_property_value
   

('nestready',
 'b942c5ec-0f97-11ea-9fd7-42010a8a0004',
 'pike',
 'MS',
 5000,
 385000,
 1,
 4,
 'other')

In [156]:
# Json is missing one or more mandatory parameters | RETURN WITH ERROR MESSAGE

if error_flag:
    error={'error_message':[error_message]}
    print(pd.DataFrame(data=error).to_json(orient="records"))
        
else:
    print("Validation Passed")

Validation Passed


In [157]:

j_user_locations=[]
j_user_states=[]
regex = re.compile(r'[^\s\w]')

if (isinstance(g_locations_value,list)):
    for i in g_locations_value:
        j_user_locations.append(i.replace('-',' ').strip().lower())
elif(isinstance(g_locations_value,str)):
     j_user_locations.append(g_locations_value.replace('-',' ').strip().lower())
    
else:
    error={'error_message':["Invalid locations type, must be str or list"]}
    print(pd.DataFrame(data=error).to_json(orient="records"))

In [158]:
if (isinstance(g_state_value,list)):
    for i in g_state_value:
        j_user_states.append(i.replace('-',' ').strip().upper())
elif(isinstance(g_state_value,str)):
     j_user_states.append(g_state_value.replace('-',' ').strip().upper())
    
else:
    error={'error_message':["Invalid state type, must be str or list"]}
    print(pd.DataFrame(data=error).to_json(orient="records"))

In [159]:
j_user_locations,j_user_states

(['pike'], ['MS'])

In [160]:
j_user_locations[0],j_user_states[0]

('pike', 'MS')

In [161]:
j_user_locations[0],g_state_value,g_state_flag

('pike', 'MS', True)

In [162]:
agents=load_agents()

In [163]:
agents[agents.agent_name=='Shane Wooten']

,Unnamed: 0,agent_name,agent_phone,agent_email,agent_office,agent_office_id,agent_office_phone,agent_office_email,properties_count,min_beds,max_beds,min_price,max_price,property_type,areaServed,state
91254,91254,Shane Wooten,318-812-7653,wshanewooten@gmail.com,Keller Williams Parishwide,243,318-812-7653,ralph@ralphmccurdy.net,45,0,4,24900,419000,"apartment,townhouse,single family detached","delhi,marion,oak grove,monroe,ouachita,grayson,madison,west carroll,sterlington,west monroe,morehouse,caldwell,union",LA


In [164]:
if len(str((j_user_locations)).split(",")) > 1:
    '''
    call multiple locations match logic
    '''
    agents,error_flag,error_message= match_locations(agents, j_user_locations, g_state_value,g_state_flag)
else:
    '''
    call one locations match logic
    '''
    agents,error_flag,error_message= match_locations_one(agents, j_user_locations,g_state_value,g_state_flag)

In [165]:
if error_flag:
    error={'error_message':[error_message]}
    print(pd.DataFrame(data=error).to_json(orient="records"))

In [166]:
agents=match_price(agents, int(g_min_price_value), int(g_max_price_value))

In [167]:
if g_property_flag:
    agents=match_property_type(agents, g_property_value)
else:
    agents["property_type_match"]=0

In [168]:
if ( (g_min_beds_flag) & (g_max_beds_flag)):
    
    if (  (not isinstance(g_min_beds_value, (int))) | (not isinstance(g_max_beds_value, (int))) ):
        error_message='min_beds and max_beds values must be INTEGERS.'
        error={'error_message':[error_message]}
        print(pd.DataFrame(data=error).to_json(orient="records"))
                
    agents=match_beds(agents, int(g_min_beds_value), int(g_max_beds_value))
else:
    agents["beds_match"]=0


In [169]:
agents.fillna(0,inplace=True)

In [170]:
g_min_price_value + (g_min_price_value * 10)/100, g_max_price_value - (g_max_price_value * 10)/100


(5500.0, 346500.0)

In [171]:
agents['score'] = agents.location_match + agents.price_match * .60 + agents.property_type_match * .30 + agents.beds_match  * .20

In [172]:
#result = agents.nlargest(5,['score','properties_count'],keep='first')[{"agent_name","agent_phone","agent_email","agent_office_phone","agent_office_email","areaServed","state","properties_count","score",}]

result = agents.nlargest(5,['score','properties_count'],keep='first')

In [173]:
result

,Unnamed: 0,agent_name,agent_phone,agent_email,agent_office,agent_office_id,agent_office_phone,agent_office_email,properties_count,min_beds,max_beds,min_price,max_price,property_type,areaServed,state,location_match,price_match,property_type_match,beds_match,score
82379,82379,Ralph Harvey,855-456-4945,0,Listwithfreedom.Com Inc,GLISTFREE01,855-456-4945,0,294,0,9,20000,3275000,"apartment,others,townhouse,single family detached","gadsden,hall,oak ridge,hendersonville,shallotte,lake wales,warner robins,walton,atlanta,collier,mc adenville,tucker,valrico,robertson,nassau,prattville,panama city,indian river,hutchinson island,daytona beach shores,clarksville,acworth,biloxi,ocklawaha,melbourne beach,enterprise,okeechobee,palm bay,pike road,jefferson,brentwood,chilton,mebane,gibsonville,elmore,gilmer,social circle,pike,st lucie,summerfield,saucier,madison,gulf,birmingham,norcross,cape canaveral,forrest,johnston,quitman,watertown,houston,miami dade,north fort myers,citrus,volusia,ruskin,vinemont,wake,iredell,lake wylie,homestead,titusville,hernando beach,gainesville,murray,north myrtle beach,estero,san antonio,hilton head island,daphne,fulton,haywood,canton,fernandina beach,pass christian,horry,dacula,greenville,bluffton,zebulon,lynn haven,rutherford,montgomery,shelby,fort myers,clemmons,cary,gastonia,mecklenburg,etowah,meridian,guntersville,milton,richland,wilson,sebastian,davidson,st helena is,columbia,knoxville,hattiesburg,thompsons station,clanton,lincoln,woodruff,pelzer,coffee,fayetteville,fort meade,chatsworth,pasco,pender,murfreesboro,lee,baldwin,cape coral,louisville,orange,tega cay,punta gorda,daytona beach,sumner,loudon,newton,fayette,clermont,morgan,durham,conyers,tuscaloosa,macon,anderson,denver,greensboro,orlando,carroll,polk,statesville,heidelberg,paulding,buford,pittsboro,port charlotte,centerville,beaufort,new smyrna beach,carrollton,manatee,mauldin,cornelius,murrells inlet,douglasville,hoover,mooresville,muscogee,greene,towns,tazewell,hillsborough,bibb,wilmington,union,winston salem,dyer,ormond beach,flowery branch,gurley,ocala,dallas,gibsonton,alamance,altamont,bonita springs,elberta,fort mill,asheboro,ridgeland,clinch,gautier,brunswick,citrus springs,spring hill,washburn,charlotte,bulloch,geneva,guilford,pawleys island,asheville,maury,marshville,spartanburg,winter haven,harrison,yancey,maryville,statesboro,senoia,marco island,naples,gulfport,rockledge,randolph,hickman,mount pleasant,jones,panama city beach,easley,blount,grundy,wetumpka,buncombe,de kalb,henderson,myrtle beach,crossville,bay,clover,pfafftown,columbus,brevard,marietta,melbourne,bradenton,pinellas,southport,old hickory,lauderdale,raleigh,tarpon springs,georgetown,forsyth,orange beach,lamar,port st joe,smyrna,hiawassee,knox,gwinnett,lawrenceville,vero beach,benton,homerville,autauga,coweta,st johns,alachua,nashville,gulf shores,chatham,york,anniston,weaverville,greenbrier,dandridge,marshall,hernando,pelham,citra,st augustine,colfax,burlington,pickens,cumberland,rocky point,cobb,jackson,grainger,chapel hill,dothan,homewood,wartrace,bedford,claiborne,gaston,high point,winter garden,newnan,calhoun,cullman,jasper,new hanover,windermere,rockdale,williamson,ellijay,clyde,st petersburg,marion,winter park,bolivia,molena,seminole,weeki wachee,calabash,gallatin,kernersville,cumming,limestone,dyersburg,burnsville,loganville","NC,GA,AL,TN,FL,SC,MS",1.0,0.0,1.0,1.0,1.5
17907,17907,Christopher Carr,610-326-2414,support@simplechoicerealty.com,"Simple Choice Realty, Inc.",SMPLCHCE,610-326-2414,support@simplechoicerealty.com,154,0,7,37000,1775000,"apartment,townhouse,single family detached","shavertown,mcmurray,clifton heights,queen annes,middletown,hockessin,westmoreland,dauphin,beaver falls,allison park,sellersville,baltimore,bucks,allegheny,mars,pike,phoenixville,gibsonia,wrightsville,upper darby,emmaus,new castle,ephrata,west chester,cecil,pulaski,bradford,berks,montgomery,schwenksville,camp hill,yeadon,rose valley,baltimore city,bridgeville,reisterstown,center valley,la

In [174]:
result_json= result.to_json(orient='records')

In [175]:
print(result_json)

[{"Unnamed: 0":82379,"agent_name":"Ralph Harvey","agent_phone":"855-456-4945","agent_email":0,"agent_office":"Listwithfreedom.Com Inc","agent_office_id":"GLISTFREE01","agent_office_phone":"855-456-4945","agent_office_email":0,"properties_count":294,"min_beds":0,"max_beds":9,"min_price":20000,"max_price":3275000,"property_type":"apartment,others,townhouse,single family detached","areaServed":"gadsden,hall,oak ridge,hendersonville,shallotte,lake wales,warner robins,walton,atlanta,collier,mc adenville,tucker,valrico,robertson,nassau,prattville,panama city,indian river,hutchinson island,daytona beach shores,clarksville,acworth,biloxi,ocklawaha,melbourne beach,enterprise,okeechobee,palm bay,pike road,jefferson,brentwood,chilton,mebane,gibsonville,elmore,gilmer,social circle,pike,st lucie,summerfield,saucier,madison,gulf,birmingham,norcross,cape canaveral,forrest,johnston,quitman,watertown,houston,miami dade,north fort myers,citrus,volusia,ruskin,vinemont,wake,iredell,lake wylie,homestead,ti

In [176]:
t=[]

In [177]:
## Inference using Real time API



## CURL COMMAND TO TEST THE API
import time

start_time = time.time()


#!curl -X POST "https://us-central1-nest-ml.cloudfunctions.net/find_realtor_v2" -H "Content-Type:application/json" --data '{"locations":["boston","dallas","roxbury","mckinney","dover","RoxburyCrossing"],"min_price":220000,"max_price":9000000}'

!curl -X POST "https://us-central1-nest-ml.cloudfunctions.net/find_realtor_v2" -H "Content-Type:application/json" --data '{"user_id":"nestready","user_key":"b942c5ec-0f97-11ea-9fd7-42010a8a0004","locations":"pike","state":"MS","min_price":95000,"max_price":385000, "min_beds":2, "max_beds":3, "property_type":"townhouse"}'



t.append(time.time() - start_time)
print("\n")
print("Second", time.time() - start_time)

[{"score":2.1,"agent_name":"Ralph Harvey","properties_count":294,"agent_office_email":0,"agent_office_phone":"855-456-4945","agent_phone":"855-456-4945","agent_office":"Listwithfreedom.Com Inc","agent_email":0},{"score":1.8,"agent_name":"LARISSA S DEAN","properties_count":2,"agent_office_email":"brentdrude@huntbrothers.com","agent_office_phone":"985-345-3344","agent_phone":"985-514-9742","agent_office":"Real Living Bobby Drude & Associates","agent_email":"ldean123@hotmail.com"},{"score":1.6,"agent_name":"ALAN J BRIDEVAUX","properties_count":3,"agent_office_email":"abridevaux@aol.com","agent_office_phone":"601-249-3400","agent_phone":"601-249-8436","agent_office":"Doug Rushing Realty, Inc.","agent_email":"abridevaux@aol.com"}]

Second 2.0070879459381104
